In [1]:
import pandas as pd
import numpy as np
import os
from PIL import Image
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras import layers, regularizers, Input, Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [2]:
train_df = pd.read_csv(r"S:\Coding\Python\MiniProject\Dataset\80_20excel\train.csv")
test_df = pd.read_csv(r"S:\Coding\Python\MiniProject\Dataset\80_20excel\test.csv")

In [3]:
le = LabelEncoder()
train_df['gender_encoded'] = le.fit_transform(train_df['gender'])
test_df['gender_encoded'] = le.transform(test_df['gender'])

In [4]:
IMG_SIZE = (128, 128)

In [5]:
def load_images(image_dir, ids):
    images = []
    for img_id in tqdm(ids):
        img_path = os.path.join(image_dir, str(img_id) + ".png")
        image = Image.open(img_path).convert('RGB')
        image = image.resize(IMG_SIZE)
        image = img_to_array(image)
        image = preprocess_input(image)
        images.append(image)
    return np.array(images)

In [6]:
train_images = load_images(r"S:\Coding\Python\MiniProject\Dataset\80_20image\train", train_df['id'])
test_images = load_images(r"S:\Coding\Python\MiniProject\Dataset\80_20image\test", test_df['id'])

100%|██████████████████████████████████████████████████████████████████████████████| 2522/2522 [01:14<00:00, 34.06it/s]


In [7]:
train_gender = train_df['gender_encoded'].values
test_gender = test_df['gender_encoded'].values

In [8]:
y_train = train_df['year'].values
y_test = test_df['year'].values

In [9]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import (Input, GlobalAveragePooling2D, Dense, Concatenate,
                                     Multiply, Add, Activation, Conv2D, Reshape)
from tensorflow.keras.models import Model
import tensorflow as tf

# CBAM block
def cbam(x, ratio=16, kernel_size=7):
    channel = x.shape[-1]

    shared_dense1 = Dense(channel // ratio, activation='relu')
    shared_dense2 = Dense(channel)

    # Channel Attention
    avg_pool = GlobalAveragePooling2D()(x)
    max_pool = tf.reduce_max(x, axis=[1, 2])  # Equivalent to GlobalMaxPooling2D

    avg_pool = Reshape((1, 1, channel))(avg_pool)
    max_pool = Reshape((1, 1, channel))(max_pool)

    avg_out = shared_dense2(shared_dense1(avg_pool))
    max_out = shared_dense2(shared_dense1(max_pool))

    channel_attention = Activation('sigmoid')(Add()([avg_out, max_out]))
    channel_refined = Multiply()([x, channel_attention])

    # Spatial Attention
    avg_spatial = tf.reduce_mean(channel_refined, axis=-1, keepdims=True)
    max_spatial = tf.reduce_max(channel_refined, axis=-1, keepdims=True)

    spatial_attention = Conv2D(1, kernel_size, padding='same', activation='sigmoid')(
        Concatenate()([avg_spatial, max_spatial])
    )

    refined = Multiply()([channel_refined, spatial_attention])
    return refined

# Image input
image_input = Input(shape=(128, 128, 3))
base_model = MobileNetV2(include_top=False, weights='imagenet', input_tensor=image_input)

# Apply CBAM after base model
x = base_model.output
x = cbam(x)  # <-- Attention added here
x = GlobalAveragePooling2D()(x)

# Metadata input
meta_input = Input(shape=(1,))
x = Concatenate()([x, meta_input])

# Dense layers
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
output = Dense(1)(x)

# Final model
model = Model(inputs=[image_input, meta_input], outputs=output)
model.compile(optimizer='adam', loss='mae', metrics=['mae'])
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][0]']           

In [10]:
checkpoint_cb = ModelCheckpoint(
    filepath=r'S:\Coding\Python\MiniProject\Models\HybridModel\epoch_{epoch:02d}_valmae_{val_mae:.2f}.h5',
    save_freq='epoch',
    save_weights_only=False,
    save_best_only=False,
    verbose=1
)
early_stop = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

In [11]:
model.fit([train_images, train_gender], y_train,
          validation_data=([test_images, test_gender], y_test),
          epochs=40,
          batch_size=4,
          callbacks=[early_stop, lr_scheduler,checkpoint_cb])

Epoch 1/40
2523/2523 [==============================] - ETA: 0s - loss: 23.2047 - mae: 23.2047       
Epoch 1: saving model to S:\Coding\Python\MiniProject\Models\HybridModel\epoch_01_valmae_23.75.h5
2523/2523 [==============================] - 89s 31ms/step - loss: 23.2047 - mae: 23.2047 - val_loss: 23.7536 - val_mae: 23.7536 - lr: 0.0010
Epoch 2/40
2521/2523 [============================>.] - ETA: 0s - loss: 15.9030 - mae: 15.9030  
Epoch 2: saving model to S:\Coding\Python\MiniProject\Models\HybridModel\epoch_02_valmae_30.64.h5
2523/2523 [==============================] - 77s 30ms/step - loss: 15.9033 - mae: 15.9033 - val_loss: 30.6409 - val_mae: 30.6409 - lr: 0.0010
Epoch 3/40
2523/2523 [==============================] - ETA: 0s - loss: 13.3104 - mae: 13.3104  
Epoch 3: saving model to S:\Coding\Python\MiniProject\Models\HybridModel\epoch_03_valmae_14.22.h5
2523/2523 [==============================] - 73s 29ms/step - loss: 13.3104 - mae: 13.3104 - val_loss: 14.2244 - val_mae: 14.22

In [12]:
model.save(r'S:\Coding\Python\MiniProject\Models\HybridModel')
model.save(r'S:\Coding\Python\MiniProject\Models\HybridModel.h5')

INFO:tensorflow:Assets written to: S:\Coding\Python\MiniProject\Models\HybridModel\assets


INFO:tensorflow:Assets written to: S:\Coding\Python\MiniProject\Models\HybridModel\assets
